In [23]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [24]:
# to see all the columns when using head() or describe()
pd.set_option('display.max_columns',60)

In [26]:
df = pd.read_csv("../diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

## For now I keep only "business and agriculture loans", "joint liability loans", "personal loans", "borrowing from an informal group", "borrowing from family and friends", "shylock loans".

In [27]:
dfb = df[((df["trx_family_code"]=="FRMLN")&((df["trx_type_code"]==2760)|(df["trx_type_code"]==3247)|(df["trx_type_code"]==2762)))|((df["trx_family_code"]=="INFGRP")&(df["trx_type_code"]==3395))]

# 1. Get rid of loans of with unexplained balance adjustments

In [28]:
dfb["trx_prx_purpose"].unique()

array(['7. Closing Balance--End of last DQ',
       '6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       'DECREASE--Unexplained balance adjustment',
       'INCREASE--Unexplained balance adjustment', '3. Payments',
       '4. Any known fees', '2. New borrowinng (individual portion)',
       '2. New borrowing', '7. Refund from lender'], dtype=object)

We insert a new column that is 0 if there is no unexplained balance adjustment for this account and a 1 if there is one. The column in 1 or 0 for all entries that correspond to that account.

In [29]:
dfb.insert(dfb.shape[1], "acc_unexplained_trx", np.zeros(dfb.shape[0]))

This program goes through all unique accounts, checks whether it has an unexplained balance adjustment and if yes sets the variable u to 1. Then it goes through another for-loop and sets the entries of that column 0 for all accounts that have an unexplained balance adjustment.

In [30]:
for acc in dfb["account_ids"].unique():
    u = 0
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        if row["trx_prx_purpose"]=="DECREASE--Unexplained balance adjustment" or row["trx_prx_purpose"]=="INCREASE--Unexplained balance adjustment":
            u = 1
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        dfb.at[index,"acc_unexplained_trx"] = u

### Let us check if it worked.

In [31]:
s = set()
for acc in dfb[dfb["acc_unexplained_trx"]==0]["account_ids"].unique():
     for i in dfb[dfb["account_ids"]==acc]["trx_prx_purpose"].unique():
            s.add(i)
print(s)

{'1. Starting balance (today)', '4. Any known fees', '2. New borrowing', '3. Payments', '5. Interest accruing', '2. New borrowinng (individual portion)', '6. Closing Balance--End of last DQ', '7. Refund from lender', '7. Closing Balance--End of last DQ'}


### "refund from lender" and "Debt written off" might be hard to deal with, but there are only 3 of those

In [32]:
dfb[dfb["trx_prx_purpose"]=="6. Debt written off"]

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx


# Let's now only use the loans that have no unexplained balance adjustments.

In [33]:
dfb = dfb[dfb["acc_unexplained_trx"]==0]

# 2. Truncation into chunks with no new borrowing

We first insert a column for the new account numbers. These will be like the old account numbers but with a .1 or .2 for each chunk after a new borrowing.

In [34]:
# this is sometimes necessary if we mess smt up
# del dfb["new_account_ids"]

In [35]:
dfb.insert(df.columns.get_loc("account_ids")+1,"new_account_ids", ["-"]*(dfb.shape[0]))

### Now let's adjust this such that new borrowings are always the first thing on one day, and then run the same program as before.

In [36]:
accs = []
for acc in dfb["account_ids"].unique():
    accs.append(acc)
    i=0
    # we will use "indices" to save the reordered indices of the dataset for each unique loan
    indices = []
    # we will use "index_bor" to save the index of the new borrowing that happens on a given day,
    #whenever there were also other transactions (since we need that index to be first)
    index_bor = 0
    #only keep the rows of that account
    lna = dfb[dfb["account_ids"]==acc]
    # sort the dataset according to time
    lna = lna.sort_values("trx_stdtime_days_acc")
    # for each day in the dataset of the account, we check whether there was 
    # a new borrowing or starting balance and if yes reorder
    for d in lna["trx_stdtime_days_acc"].unique():
        # if only one entry that day we just add the index to indices
        if lna[lna["trx_stdtime_days_acc"]==d].shape[0]==1:
            indices.append(lna[lna["trx_stdtime_days_acc"]==d].index[0])
        # if there are more than 1 entry that day
        elif lna[lna["trx_stdtime_days_acc"]==d].shape[0]>1:
            # if there are more than one new borrowings or starting balances, we need to check manually
            if lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]>1:
                print("Several new borrowings on day " + str(d) +", check manually account " + acc + " in category "+ str(lna["trx_type_code"].unique()))
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                            indices.append(i)
            #if there is exactly one new borrowing or starting balance that day (here we will need to check for each loan type what they exactly call new borrowings)
            elif lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]==1:
                #for each type of new borrowing if there is one that day, put that index first into indices
                for kind in ["2. New borrowing","2. New borrowinng (individual portion)","1. Starting balance (today)"]:
                    if kind in lna[lna["trx_stdtime_days_acc"]==d]["trx_prx_purpose"].unique():
                        index_bor = lna[(lna["trx_stdtime_days_acc"]==d) & (lna["trx_prx_purpose"]==kind)].index[0]
                        indices.append(index_bor)
                        #then add all other indices of that day
                        for i in lna[lna["trx_stdtime_days_acc"]==d].index.drop(index_bor):
                            indices.append(i)
            # if there are several entries for that day but none of them new borrowings just add all indices of that day            
            else:
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                    indices.append(i)
    # here we reindex the lna dataset
    lna = lna.loc[indices,]
    #v will be the index of the chunk we are currently at
    v=0
    # saving the old account number
    acc = lna["account_ids"].unique()[0]
    # a is the new account number for each chunk
    a = lna["account_ids"].unique()[0]+".0"
    # we go through all rows of lna and change the entries of the column "new_account_ids" in the original dataset.
    for index, row in lna.iterrows():
        # if there is a new borrowing in the row, increase v by 1 and set a to acc.v
        if row["trx_prx_purpose"] == "1. Starting balance (today)" or row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
            v +=1
            a = acc+"."+str(v)
            dfb.at[index,"new_account_ids"]=a
        # otherwise set the new account number in that row to a
        else:
             dfb.at[index,"new_account_ids"]=a

Several new borrowings on day 244, check manually account 61135946960100000 in category [3395]


Several borrowings on the same day.
57137225366300000
61135946960100000
61134787763500000

In [37]:
df[df["account_ids"]=="57137225366300000"].sort_values("trx_stdtime_days_acc")[["trx_type_desc","trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_type_desc,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
476372,Friends and family: Borrowing,0,2. New borrowing,1000.0,Increase
475250,Friends and family: Borrowing,4,2. New borrowing,1000.0,Increase
473295,Friends and family: Borrowing,6,3. Payments,2000.0,Decrease
476508,Friends and family: Borrowing,14,2. New borrowing,1300.0,Increase
475933,Friends and family: Borrowing,38,3. Payments,1300.0,Decrease
475125,Friends and family: Borrowing,175,2. New borrowing,200.0,Increase
475997,Friends and family: Borrowing,182,2. New borrowing,500.0,Increase
474926,Friends and family: Borrowing,188,3. Payments,700.0,Decrease
475005,Friends and family: Borrowing,203,2. New borrowing,600.0,Increase
474162,Friends and family: Borrowing,220,3. Payments,600.0,Decrease


In [38]:
df[df["account_ids"]=="61134787763500000"][["trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
17354,0,1. Starting balance (today),0.0,Increase
22833,0,1. Starting balance (today),0.0,Increase


The account 61135946960100000 seems to have several new borrowings on day 244.

In [39]:
df[df["account_ids"]=="61135946960100000"][["trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
5052,285,7. Closing Balance--End of last DQ,3800.0,NaN
43429,141,2. New borrowing,2000.0,Increase
118905,0,2. New borrowing,500.0,Increase
195059,5,3. Payments,500.0,Decrease
212855,244,2. New borrowing,1300.0,Increase
295268,244,2. New borrowing,500.0,Increase


### Should we just get rid of these accounts?

In [40]:
dfb = dfb[(dfb["account_ids"]!="57137225366300000")&(dfb["account_ids"]!="61135946960100000")&(dfb["account_ids"]!="61134787763500000")]

## Let's check if it worked.

Let's see if all accounts got a new name.

In [41]:
dfb[dfb["new_account_ids"]=="-"]

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx


In [42]:
df = dfb

In [43]:
#add balance evolution to original dataset
df.insert(df.shape[1],"bal_evol",np.zeros(df.shape[0]))

In [44]:
#add column for starting balance 
df.insert(df.shape[1],"start_bal",np.zeros(df.shape[0]))

In [45]:
#see the purposes for which there is no increase or decrease
exc = set()
for acc in df["new_account_ids"].unique():
    lna = df[df["new_account_ids"]==acc]
    #we want the lna dataset with the new borrowing the first entry of the first day
    #we reuse the algorithm from the truncation program.
    # sort the dataset according to time
    i=0
    # we will use "indices" to save the reordered indices of the dataset for each unique loan
    indices = []
    # we will use "index_bor" to save the index of the new borrowing that happens on a given day,
    #whenever there were also other transactions (since we need that index to be first)
    index_bor = 0
    # sort the dataset according to time
    lna = lna.sort_values("trx_stdtime_days_acc")
    # for each day in the dataset of the account, we check whether there was 
    # a new borrowing or starting balance and if yes reorder
    for d in lna["trx_stdtime_days_acc"].unique():
        # if only one entry that day we just add the index to indices
        if lna[lna["trx_stdtime_days_acc"]==d].shape[0]==1:
            indices.append(lna[lna["trx_stdtime_days_acc"]==d].index[0])
        # if there are more than 1 entry that day
        elif lna[lna["trx_stdtime_days_acc"]==d].shape[0]>1:
            # if there are more than one new borrowings or starting balances, we need to check manually
            if lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]>1:
                print("Several new borrowings on day " + str(d) +", check manually account " + acc + " in category "+ str(lna["trx_type_code"].unique()))
            #if there is exactly one new borrowing or starting balance that day (here we will need to check for each loan type what they exactly call new borrowings)
            elif lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]==1:
                #for each type of new borrowing if there is one that day, put that index first into indices
                for kind in ["2. New borrowing","2. New borrowinng (individual portion)","1. Starting balance (today)"]:
                    if kind in lna[lna["trx_stdtime_days_acc"]==d]["trx_prx_purpose"].unique():
                        index_bor = lna[(lna["trx_stdtime_days_acc"]==d) & (lna["trx_prx_purpose"]==kind)].index[0]
                        indices.append(index_bor)
                        #then add all other indices of that day
                        for i in lna[lna["trx_stdtime_days_acc"]==d].index.drop(index_bor):
                            indices.append(i)
            # if there are several entries for that day but none of them new borrowings just add all indices of that day            
            else:
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                    indices.append(i)
    # here we reindex the lna dataset
    lna = lna.loc[indices,]
    
    #add transaction with direction for account
    lna.insert(58, "trx_val_dir",np.zeros(lna.shape[0]))
    for index, row in lna.iterrows():
        if lna.loc[index,"trx_bsheet_direction"]=="Increase":
            lna.at[index,"trx_val_dir"] = lna.loc[index,"trx_value_kes"]
        elif lna.loc[index,"trx_bsheet_direction"]=="Decrease":
            lna.at[index,"trx_val_dir"] = -lna.loc[index,"trx_value_kes"]
        else:
            #save where there is neither increase nor decrease
            exc.add(row["trx_prx_purpose"])
    
    #add the balance evolution entries to df 
    s=0
    start_bal = lna.iloc[0,df.columns.get_loc("trx_value_kes")]
    for index,row in lna.iterrows():
        s+=lna.loc[index,"trx_val_dir"]
        df.at[index,"bal_evol"]=s
        df.at[index,"start_bal"]=start_bal
    #add the starting balance to df
    
    

In [46]:
df = df[df["start_bal"]>10]

In [47]:
df.to_csv("diaries_trx_trunc_loans_start_bal.csv")

### Careful: the column "unique_accnts" is not accurate anymore.

In [48]:
df.sample(5)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,...,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx,bal_evol,start_bal
455789,KNBOK03,NaN,01nov2012,22oct2013,355,12,04=Diaries Interview,25jan2013,1,2013,2013_01,21nov2012,89135780169200000,89135780169200000.1,NaN,64134408292500000,NaN,Liability,NaN,Informal,NaN,30dec2012,22oct2013,296,10,89137653955100000,64134408292500000,20jan2013,1,2013,...,80,2,21,0,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,3. Payments,Repayment on financial liability,0.0,Decrease,4.0,"Mobile money transfer (M-PESA, Airtel Money, O...",0.0,6.082738,04=Mobile phone only,Outflow,1070.0,12.588230,0,NaN,NaN,NaN,2.758621,0.0,14860.0,20000.0
116501,KMOMB06,NaN,03sep2012,30dec2013,483,16,04=Diaries Interview,29oct2012,10,2012,2012_10,21aug2012,57134676338900000,57134676338900000.2,NaN,57134294208000000,NaN,Liability,NaN,Informal,NaN,04sep2012,19may2013,257,8,57135149704700000,57134294208000000,25oct2012,10,2012,...,52,1,51,1,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,500.0,5.882353,0,NaN,NaN,NaN,1.793103,0.0,12300.0,12800.0
461674,KNBOK41,NaN,08nov2012,05nov2013,362,12,04=Diaries Interview,05feb2013,2,2013,2013_02,16nov2012,50135506816000000,50135506816000000.1,NaN,50135271114900000,NaN,Liability,NaN,Informal,NaN,26nov2012,05nov2013,344,11,50136663150900000,50135271114900000,30jan2013,1,2013,...,83,2,65,2,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,0.0,2.088910,"01=Bank, MFI, SACCO, insurance branch",Outflow,6700.0,78.823530,0,NaN,NaN,NaN,2.862069,0.0,71000.0,98000.0
380580,KMAKE03,NaN,01oct2012,25sep2013,359,12,04=Diaries Interview,03jul2013,7,2013,2013_07,03sep2012,63137323837500000,63137323837500000.1,NaN,63134450468300000,NaN,Liability,NaN,Informal,NaN,21may2013,25sep2013,127,4,63137324436700000,63134450468300000,25jun2013,6,2013,...,267,9,35,1,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,300.0,3.529412,0,NaN,NaN,NaN,9.206897,0.0,0.0,300.0
44685,KVIHC16,NaN,24sep2012,19sep2013,360,12,04=Diaries Interview,08oct2012,10,2012,2012_10,20aug2012,59134803803300000,59134803803300000.1,1.0,59134423954300000,NaN,Liability,NaN,Informal,NaN,07oct2012,19sep2013,347,11,59135036550800000,59134423954300000,07oct2012,10,2012,...,13,0,0,0,findev,"Borrowing, lending, savings or insurance media...",INFGRP,Informal group,3395,Borrowing from an informal Group,2. New borrowing,New borrowing on financial liability,0.0,Increase,1.0,Cash,1.0,0.000000,07=Group meeting,Inflow,500.0,5.882353,0,NaN,NaN,NaN,0.448276,0.0,500.0,500.0
